Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.3912, -0.1520,  0.1454, -0.3617, -0.0353,  0.0252],
        [ 0.3039, -0.0041,  0.0484,  0.3044,  0.1016, -0.3819],
        [ 0.1601,  0.3536,  0.0356,  0.0489,  0.2322, -0.1651],
        [-0.0689, -0.3283, -0.0093, -0.1547,  0.0790, -0.0387],
        [ 0.2779,  0.0771,  0.3021,  0.0324,  0.2329,  0.3972],
        [ 0.1122,  0.2093, -0.1448, -0.1591, -0.2136,  0.3781],
        [ 0.1878, -0.3351,  0.4009, -0.1552, -0.0425, -0.2517],
        [ 0.1755, -0.3864,  0.3829,  0.2507, -0.1042,  0.0345],
        [ 0.3790,  0.0832, -0.1289,  0.0797,  0.3000, -0.1455],
        [-0.1481,  0.0072,  0.1873, -0.3804, -0.3036,  0.1707],
        [ 0.2921,  0.0815, -0.3335, -0.3416,  0.2847, -0.0560],
        [ 0.1649, -0.1533,  0.2353, -0.2611,  0.2912, -0.1699],
        [-0.2406,  0.1268, -0.2732,  0.4017,  0.3174,  0.0952],
        [-0.1574,  0.3119, -0.3984, -0.3331,  0.2092,  0.2088],
        [ 0.0173,  0.1627, -0.0118, -0.3936, -0.0588, -0.0288],
        [ 0.1400, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.1642],
        [0.1631],
        [0.1617],
        ...,
        [0.1639],
        [0.1622],
        [0.1629]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0468)
1 tensor(0.0463)
2 tensor(0.0458)
3 tensor(0.0455)
4 tensor(0.0451)
5 tensor(0.0449)
6 tensor(0.0447)
7 tensor(0.0445)
8 tensor(0.0443)
9 tensor(0.0441)
10 tensor(0.0440)
11 tensor(0.0438)
12 tensor(0.0436)
13 tensor(0.0435)
14 tensor(0.0432)
15 tensor(0.0430)
16 tensor(0.0428)
17 tensor(0.0425)
18 tensor(0.0422)
19 tensor(0.0420)
20 tensor(0.0417)
21 tensor(0.0414)
22 tensor(0.0411)
23 tensor(0.0407)
24 tensor(0.0404)
25 tensor(0.0401)
26 tensor(0.0398)
27 tensor(0.0395)
28 tensor(0.0391)
29 tensor(0.0388)
30 tensor(0.0384)
31 tensor(0.0380)
32 tensor(0.0376)
33 tensor(0.0372)
34 tensor(0.0368)
35 tensor(0.0363)
36 tensor(0.0358)
37 tensor(0.0354)
38 tensor(0.0349)
39 tensor(0.0343)
40 tensor(0.0338)
41 tensor(0.0333)
42 tensor(0.0327)
43 tensor(0.0321)
44 tensor(0.0315)
45 tensor(0.0309)
46 tensor(0.0303)
47 tensor(0.0297)
48 tensor(0.0290)
49 tensor(0.0284)
50 tensor(0.0277)
51 tensor(0.0270)
52 tensor(0.0263)
53 tensor(0.0256)
54 tensor(0.0248)
55 tensor(0.0241)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1469.80177402]
[1409.99999118] [1458.12092924]
[1128.99997044] [1166.39292288]
[12991.99967575] [11751.21705246]
[14720.0001564] [14515.54244614]
[2938.99994564] [2733.24961567]
[6635.00014687] [8059.33972931]
[11215.99973297] [9306.47686005]
[1040.99999189] [1427.81092644]
[5040.00003624] [5214.84655762]
[9541.99988747] [8460.72212601]
[3792.99988461] [5434.98217392]
[1002.0000155] [749.33661652]
[2542.00005341] [1533.65360451]
[6788.00001717] [6748.97502899]
[9386.00025749] [7768.26602554]
[1245.99996853] [1723.54274654]
[997.00000262] [514.48001862]
[708.00000262] [731.48826694]
[1435.99997544] [1362.11525249]
[2590.00006676] [2507.21190882]
[1392.99997497] [1636.27217484]
[11755.99946976] [9536.82885742]
[2007.99997377] [1296.99698544]
[2707.99998474] [2678.1740942]
[631.00000417] [774.23539352]
[652.99999881] [1001.90982151]
[2492.00006247] [2712.58396149]
[594.99999416] [521.78180408]
[760.0000056] [1067.57710743]
[1179.00003028] [1055.34572792]
[1789.9999361] [1

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [911.77729607]
[628.00000334] [706.73391056]
[442.00000328] [715.10810184]
[2284.99997044] [2642.59838867]
[1040.99999189] [835.5323019]
[809.99999654] [1222.47401428]
[3049.99987316] [3368.86572742]
[720.00000596] [659.9148941]
[877.99999249] [596.59086037]
[1292.00000429] [1574.86750221]
[7056.00004578] [4796.33102417]
[4176.99999142] [5879.05188942]
[13701.00020599] [14336.07391357]
[2948.0000515] [3964.0666647]
[16732.99985504] [14233.00499344]
[1815.00000048] [2051.22542477]
[16220.00016022] [12066.37482452]
[5967.00019073] [4729.50837326]
[473.00000042] [621.91242599]
[1078.9999795] [1037.29507923]
[452.00000319] [598.38206196]
[801.99998283] [789.91605568]
[2458.99994993] [2550.35605431]
[2492.99998236] [2742.94371223]
[582.00000203] [674.81998444]
[1405.9999671] [1404.89352322]
[8715.99991035] [11572.92593384]
[7634.00004578] [6652.17793274]
[506.99999839] [609.75380802]
[665.99999094] [728.41547012]
[1815.00000048] [1580.59030724]
[707.00001383] [607.75892925]
[

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1381.1299083762701


## Conclusão



Nesse notebook fizemos uma rede neural MLP otimizada que é mais complexa e versátil, com o pytorch, que utiliza o método Adam de otimização para treinar, o que ajuda a rede a fazer a melhor previsão com o passar das épocas.

Em alguns tópicos de discussões que acho importante;
- Quando precisarmos utilizarmos redes neurais na vida, não será necessário fazer uma do 0, afinal existem muitas redes prontas na internet, apenas vamos aplicá-las aos nossos dados.
- Método de otimização melhoram o treino das redes.
- As redes gostam de dados que variam entre 0 e 1 ou -1 e 1.
- O loss diminui conforme os treinos são feitos, então temos que ter cuidado com overfitting de dados, o que nos leva a ter que buscar um número ideal de épocas com outros métodos com dados de validação, ou seja, pegaremos a quantidade de épocas através do ponto mínimo dos dados de validação.
- Quanto mais você treina a rede (milhares de épocas) melhor a previsão ficará.

## Playground

